# First Lookup on DemoParse Library


In [34]:
from demoparser2 import DemoParser
import pandas as pd
from pprint import pprint
from typing import Dict, Sequence, Optional, List, Tuple
import numpy as np

# from utils.scoreboard_info import StatsCalculator

In [35]:
base_path = "../../demos"
parser = DemoParser(
    "/Users/luneto10/Documents/Exploratory/CS2_Stats/demos/gc/pulin-gc.dem"
)

event_names = parser.list_game_events()

# Currently the event "all" gives you all events. Cursed solution for now
event_names

['round_prestart',
 'bomb_planted',
 'weapon_reload',
 'player_footstep',
 'buytime_ended',
 'round_officially_ended',
 'hltv_versioninfo',
 'round_announce_match_start',
 'round_announce_match_point',
 'inferno_startburn',
 'player_team',
 'cs_win_panel_match',
 'round_freeze_end',
 'hltv_chase',
 'cs_round_final_beep',
 'hltv_fixed',
 'inferno_expire',
 'announce_phase_end',
 'cs_round_start_beep',
 'player_blind',
 'round_announce_final',
 'bomb_defused',
 'server_cvar',
 'player_connect_full',
 'round_announce_last_round_half',
 'bomb_dropped',
 'bomb_beginplant',
 'player_hurt',
 'smokegrenade_detonate',
 'item_pickup',
 'flashbang_detonate',
 'round_poststart',
 'chat_message',
 'player_disconnect',
 'weapon_zoom',
 'cs_pre_restart',
 'hegrenade_detonate',
 'round_time_warning',
 'player_jump',
 'bomb_pickup',
 'other_death',
 'player_connect',
 'player_death',
 'player_spawn',
 'begin_new_match',
 'smokegrenade_expired',
 'weapon_fire',
 'item_equip',
 'bomb_begindefuse',
 'bomb

# Game Score

Halftime or final score


def get_player_average_damage_per_round():
    """
    Function to calculate the Average Damage per Round (ADR) for all players.

    Args:
        parser (DemoParser): An instance of DemoParser initialized with a demo file path.

    Returns:
        pd.DataFrame: A dataframe containing player Steam IDs and their ADR.
    """
    # Define the fields to extract
    wanted_fields = ["damage_total", "kills_total", "deaths_total"]

    # Parse the number of rounds
    round_end_df = parser.parse_event("round_end")
    return len(round_end_df) - 1  # Count the total number of rounds

    # # Parse the maximum tick to analyze final stats
    # max_tick = round_end_df["tick"].max()

    # # Parse the wanted fields for all players at the last tick
    # stats_df = parser.parse_ticks(wanted_fields, ticks=[max_tick])

    # # Ensure no NaN values in the stats
    # stats_df.fillna(0, inplace=True)

def get_final_score(
    players: Sequence[str] = None,
    round_info: str | int = "final",
) -> Tuple[pd.DataFrame, pd.DataFrame] | pd.DataFrame:
    """
    Retrieve the final score details for specified players or all players at the end of a round.

    Parameters:
    -----------
    players : Sequence[str], optional
        A list of player names to filter the DataFrame. If provided, the returned DataFrame
        will only include data for the specified players. Defaults to None.

    round_info : str | int, optional
        Specifies the round for which to retrieve score data. Can take the following values:
        - "final" (default): Retrieves the final round score.
        - "half_time": Retrieves the score at halftime (end of round 12).
        - An integer: Retrieves the score for the specified round (e.g., 1 for the first round).
        If the integer is greater than the maximum round available, a ValueError is raised.

    Returns:
    --------
    Union[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame], pd.DataFrame]
        - If `players` is provided: A single filtered DataFrame containing data for the specified players.
        - Otherwise: A tuple containing:
            1. DataFrame for Team 1
            2. DataFrame for Team 2
            3. DataFrame containing data for all players, sorted by team and KD ratio.

    Raises:
    -------
    ValueError
        - If `round_info` is not "final", "half_time", or a valid integer.
        - If `round_info` is an integer greater than the maximum round available.
    """

    # Determine the tick based on round_info
    last_tick = parser.parse_event("round_end")['tick'].max()

    events = pd.concat([
        parser.parse_event("round_officially_ended")["tick"].drop_duplicates(),
        pd.Series([last_tick])
    ], ignore_index=True)

    events.index = range(1, len(events) + 1)

    max_round = len(events)

    # Validate and process `round_info`
    special_rounds = {
    "half_time": 12,
    "final": max_round
    }

    # Validate and process `round_info`
    if isinstance(round_info, int):
        if not 1 <= round_info <= max_round:
            raise ValueError(f"Invalid `round_info`: {round_info}. Maximum round is {max_round}.")
    elif isinstance(round_info, str) and round_info in special_rounds:
        round_info = special_rounds[round_info]
    else:
        raise ValueError("Invalid `round_info`. Must be 'final', 'half_time', or an integer.")


    # Get the tick corresponding to `round_info`
    tick = events.loc[round_info]

    wanted_fields = [
        "kills_total",
        "deaths_total",
        "mvps",
        "headshot_kills_total",
        "ace_rounds_total",
        "4k_rounds_total",
        "3k_rounds_total",
        "team_num",
        "damage_total",
        "assists_total",
        "team_score_first_half",
        "team_score_second_half",
    ]

    # Parse the ticks at the max_tick
    df = parser.parse_ticks(wanted_fields, ticks=[tick])
    df["deaths_total"] = df["deaths_total"].fillna(0)

    # Calculate KD
    df["kd"] = np.where(
        df["deaths_total"] != 0,
        round(df["kills_total"] / df["deaths_total"], 2),
        round((df["kills_total"] / 1), 2),
    )

    # Calculate HS %
    df["headshot_percentage"] = np.where(
        df["kills_total"] != 0,
        round(df["headshot_kills_total"] / df["kills_total"] * 100),
        round((df["headshot_kills_total"] / 1) * 100),
    ).astype(int)

    # Calculate ADR
    print(df["damage_total"])
    df["adr"] = round(df["damage_total"] / round_info, 2)
    df["kpr"] = round(df["kills_total"] / round_info, 2)
    df["dpr"] = round(df["deaths_total"] / round_info, 2)
    
    df["round"] = round_info

    df["diff"] = df["kills_total"] - df["deaths_total"]

    df.sort_values("adr", inplace=True, ascending=False)

    # If specific players are provided, filter the DataFrame and return
    if players:
        df = df[df["name"].isin(players)]
        return df

    # Get unique team numbers dynamically
    unique_teams = df["team_num"].unique()

    # Assign the team numbers dynamically
    team_num_1, team_num_2 = unique_teams[:2]

    # Create separate DataFrames for each team
    df_team_1: pd.DataFrame = df[df["team_num"] == team_num_1].copy()
    df_team_2: pd.DataFrame = df[df["team_num"] == team_num_2].copy()

    return df_team_1, df_team_2, df


df = get_final_score(round_info=23)
df[2]

In [36]:
# wanted_fields = [
#         "kills_total",
#         "deaths_total",
#         "assists_total",
#         "damage_total",
#         "team_score_first_half",
#         "team_score_second_half",
#     ]

#     # Parse the ticks at the max_tick
# ticks = parser.parse_event("round_end")["tick"]
# df = parser.parse_ticks(wanted_fields, ticks=[165813])
# df
# # for tick in ticks:
    
# #     df['tick'] = tick
# #     df = df[df['steamid'] == 76561199075107764]
# #     print(df)


In [37]:
# last_tick = parser.parse_event("round_end")['tick'].max()
# events = parser.parse_event("round_officially_ended")["tick"].drop_duplicates().reset_index(drop=True)
# events = pd.concat([events, pd.Series([last_tick])], ignore_index=True)
# events.index = range(1, len(events) + 1)

# # print(events)

In [38]:
wanted_fields = [
        "kills_total",
        "deaths_total",
        "assists_total",
        "damage_total",
        "team_score_first_half",
        "team_score_second_half",
    ]

    # Parse the ticks at the max_tick
ticks = parser.parse_event("round_end")["tick"]
df = parser.parse_ticks(wanted_fields, ticks=[120574])
df

,kills_total,deaths_total,assists_total,damage_total,team_score_first_half,team_score_second_half,tick,steamid,name
0,12,12,5,1425,9,1,120574,76561199075107764,✓ ★ ⑳ twitch.tv/PuliNFPS
1,15,16,5,1622,9,1,120574,76561198335553425,✓ ☆ ⑳ tt.tv/apreciem LIVE ON
2,25,11,5,2323,3,6,120574,76561198848991940,✓ ⑳ felicio999-
3,12,14,6,1440,3,6,120574,76561199123158663,✓ ☆ ⑳ t.tv/vilacattv
4,13,11,3,1363,9,1,120574,76561198341120471,✓ ⑳ calladin
5,11,15,7,1588,3,6,120574,76561199215548394,✓ ☆ ⑳ mknfps
6,9,12,1,985,9,1,120574,76561198430796631,✓ ☆ ⑯ forbbiden-_-
7,16,14,4,1864,9,1,120574,76561198278676389,✓ ★ ⑳ Ryypher
8,8,10,5,969,3,6,120574,76561198321177508,✓ ⑰ gnomadas kit bota
9,8,15,3,1105,3,6,120574,76561198350357012,★ ⑳ luiza monza


In [39]:
from collections import defaultdict
from demoparser2 import DemoParser
from functools import lru_cache
import numpy as np
from typing import Sequence, Union, Tuple
import pandas as pd

from utils.interface.parserInterface import ParserInterface


class StatsCalculator:
    def __init__(self, parser: ParserInterface) -> None:
        """
        Initialize the FinalScoreCalculator with a parser instance.

        Parameters:
        -----------
        parser : object
            The parser instance used to retrieve event and tick data.
        """
        self.parser = parser

    @lru_cache
    def __get_tick_for_round(self, round_info: Union[str, int]) -> Tuple[int, int]:
        """
        Retrieve the tick corresponding to the specified round.

        Parameters:
        -----------
        round_info : str | int
            Specifies the round. Can be:
            - "final": Retrieves the final round tick.
            - "half_time": Retrieves the halftime round tick (end of round 12).
            - An integer: Retrieves the tick for the specified round.

        Returns:
        --------
        int
            The tick for the specified round.

        Raises:
        -------
        ValueError:
            If `round_info` is invalid or exceeds the maximum round.
        """
        # Determine the tick based on round_info
        last_tick = self.parser.parse_event("round_end")["tick"].max()

        events = pd.concat(
            [
                self.parser.parse_event("round_officially_ended")[
                    "tick"
                ].drop_duplicates(),
                pd.Series([last_tick]),
            ],
            ignore_index=True,
        )

        events.index = range(1, len(events) + 1)

        max_round = len(events)

        special_rounds = {"half_time": 12, "final": max_round}

        # Validate and process `round_info`
        if isinstance(round_info, int):
            if not 1 <= round_info <= max_round:
                raise ValueError(
                    f"Invalid `round_info`: {round_info}. Maximum round is {max_round}."
                )
        elif isinstance(round_info, str) and round_info in special_rounds:
            round_info = special_rounds[round_info]
        else:
            raise ValueError(
                "Invalid `round_info`. Must be 'final', 'half_time', or an integer."
            )

        # Get the tick corresponding to `round_info`
        return events.loc[round_info], round_info

    def get_total_rounds(self, platform: str) -> int:
        """
        Retrieve the total number of rounds played.

        Returns:
        --------
        int
            The total number of rounds played.
        """
        result = len(
            self.parser.parse_event("round_officially_ended")["tick"].drop_duplicates()
        )
        if platform == "gc":
            return result - 1
        elif platform == "faceit":
            return result + 1
        elif platform == "mm":
            return result
        else:
            raise ValueError(f"Invalid platform: {platform}")

    def __calculate_metrics(self, df: pd.DataFrame, actual_rounds: int) -> pd.DataFrame:
        """
        Calculate performance metrics for the given DataFrame.

        Parameters:
        -----------
        df : pd.DataFrame
            The DataFrame containing player statistics.

        actual_rounds : int
            The number of rounds played.

        Returns:
        --------
        pd.DataFrame
            The DataFrame with additional calculated metrics.
        """
        df["deaths_total"] = df["deaths_total"].fillna(0)

        # KD Ratio
        df["kd"] = np.where(
            df["deaths_total"] != 0,
            round(df["kills_total"] / df["deaths_total"], 2),
            df["kills_total"],
        )

        # Headshot Percentage
        df["headshot_percentage"] = np.where(
            df["kills_total"] != 0,
            round(df["headshot_kills_total"] / df["kills_total"] * 100),
            0,
        ).astype(int)

        # ADR, KPR, and DPR
        df["adr"] = round(df["damage_total"] / actual_rounds, 2)
        df["kpr"] = round(df["kills_total"] / actual_rounds, 2)
        df["dpr"] = round(df["deaths_total"] / actual_rounds, 2)

        # Kill-Death Difference
        df["diff"] = df["kills_total"] - df["deaths_total"]
        df["round"] = actual_rounds

        return df

    def __split_by_team(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Split the DataFrame into two separate DataFrames for each team.

        Parameters:
        -----------
        df : pd.DataFrame
            The DataFrame to split.

        Returns:
        --------
        Tuple[pd.DataFrame, pd.DataFrame]
            A tuple containing the DataFrames for Team 1 and Team 2.
        """
        unique_teams = df["team_num"].unique()
        if len(unique_teams) < 2:
            raise ValueError("Insufficient teams in the data.")

        team_num_1, team_num_2 = unique_teams[:2]
        df_team_1 = df[df["team_num"] == team_num_1].copy()
        df_team_2 = df[df["team_num"] == team_num_2].copy()

        return df_team_1, df_team_2

    def get_scoreboard(
        self, players: Sequence[str] = None, round_info: Union[str, int] = "final"
    ) -> Union[pd.DataFrame, Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]]:
        """
        Retrieve the scoreboard details for specified players or all players at a given round.

        Parameters:
        -----------
        players : Sequence[str], optional
            A list of player names to filter the DataFrame.

        round_info : Union[str, int], optional
            Specifies the round to retrieve score data for.

        Returns:
        --------
        Union[pd.DataFrame, Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]]
            - If `players` is provided: A single filtered DataFrame.
            - Otherwise: A tuple containing:
                1. DataFrame for Team 1
                2. DataFrame for Team 2
                3. DataFrame for all players, sorted by ADR.
        """
        tick, actual_rounds = self.__get_tick_for_round(round_info)

        wanted_fields = [
            "kills_total",
            "deaths_total",
            "mvps",
            "headshot_kills_total",
            "ace_rounds_total",
            "4k_rounds_total",
            "3k_rounds_total",
            "team_num",
            "damage_total",
            "assists_total",
            "team_score_first_half",
            "team_score_second_half",
        ]

        # Parse the ticks
        df = self.parser.parse_ticks(wanted_fields, ticks=[tick])

        # Calculate metrics
        df = self.__calculate_metrics(df, actual_rounds)

        # Sort by ADR
        df.sort_values("adr", inplace=True, ascending=False)

        # If players are provided, filter the DataFrame
        if players:
            return df[df["name"].isin(players)]

        # Split by team and return
        df_team_1, df_team_2 = self.__split_by_team(df)
        return df_team_1, df_team_2, df

    def __get_round_interval_ticks(self):
        result = []
        max_round = self.get_total_rounds() + 1
        df_start = parser.parse_event("round_start")
        df_end = parser.parse_event("round_end")["tick"]
        for i in range(1, max_round):
            round_start = df_start.query(f"round == {i}")["tick"].max()
            round_end = df_end[i]
            result.append((round_start, round_end))
        return result

    def get_first_kills(self) -> pd.DataFrame:
        """
        Analyze the first kill for each round and return detailed information.

        Returns:
        --------
        pd.DataFrame:
            DataFrame containing details of the first kill for each round.
        """
        df = self.parser.parse_event("player_death")
        round_interval = self.__get_round_interval_ticks()

        # Filter valid ticks
        df = df[df["tick"] >= round_interval[0][0]]

        # Initialize data storage
        round_first_kill = defaultdict(
            lambda: {"attacker_name": "", "rounds": [], "amount": 0, "killed": []}
        )

        # Iterate over rounds
        for round_number in range(self.get_total_rounds()):
            round_df: pd.DataFrame = df[
                (df["tick"] >= round_interval[round_number][0])
                & (df["tick"] <= round_interval[round_number][1])
            ]

            if round_df.empty:
                continue

            first_kill = round_df.nsmallest(1, "tick")[
                ["attacker_name", "attacker_steamid", "user_name"]
            ].values[0]
            attacker_id = first_kill[1]

            round_first_kill[attacker_id]["rounds"].append(round_number + 1)
            round_first_kill[attacker_id]["killed"].append(first_kill[2])
            round_first_kill[attacker_id]["amount"] += 1
            round_first_kill[attacker_id]["attacker_name"] = first_kill[0]

        # Convert to DataFrame
        result_df = pd.DataFrame.from_dict(round_first_kill, orient="index")
        result_df.index.name = "attacker_steamid"
        return result_df.reset_index()

In [56]:
# pprint(parser.parse_event("round_officially_ended").drop_duplicates())
parser = DemoParser(
        "/Users/luneto10/Documents/Exploratory/CS2_Stats/demos/gc/pulin-gc.dem"
    )
# parser = DemoParser(
#         "/Users/luneto10/Documents/Exploratory/CS2_Stats/demos/faceit/anubisFaceit.dem"
    # )
scoreboard = StatsCalculator(parser)
# scoreboard.get_total_rounds("faceit")
# pprint(parser.parse_event("round_officially_ended")["tick"].drop_duplicates())
print(parser.parse_event("server_message"))
# parser.parse_ticks(wanted_fields, ticks=[17555])

[]
